In [15]:
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import open3d as o3d

In [16]:
img_color = cv2.imread('test_imgs/color_new.png')
img_depth = cv2.imread('test_imgs/depth_new.png', 0)
print('color shape: ', img_color.shape)
print('depth shape: ', img_depth.shape)

color shape:  (480, 640, 3)
depth shape:  (480, 640)


In [17]:
height, width = img_color.shape[:2] # 이미지의 높이와 너비 불러옴, 가로 [0], 세로[1]

img_hsv = cv2.cvtColor(img_color, cv2.COLOR_BGR2HSV) # cvtColor 함수를 이용하여 hsv 색공간으로 변환

lower_violet = (125-20, 30, 30)
upper_violet = (125+20, 255, 255)

# # lower boundary RED color range values; Hue (0 - 10)
# lower_red1 = np.array([0, 75, 100])
# upper_red1 = np.array([10, 120, 255])

# # upper boundary RED color range values; Hue (160 - 180)
# lower_red2 = np.array([150,75,100])
# upper_red2 = np.array([179,120,255])

# mask_red1 = cv2.inRange(img_hsv, lower_red1, upper_red1)
# mask_red2 = cv2.inRange(img_hsv, lower_red2, upper_red2)
# img_mask = mask_red1 + mask_red2

img_mask = cv2.inRange(img_hsv, lower_violet, upper_violet) # 범위내의 픽셀들은 흰색, 나머지 검은색

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
erosion_image = cv2.erode(img_mask, kernel, iterations=2)  #// make erosion image
img_mask = cv2.dilate(erosion_image, kernel, iterations=2)  #// make dilation image


In [18]:
# 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
img_result = cv2.bitwise_and(img_color, img_color, mask = img_mask) 

In [19]:
cv2.imshow('img_color', img_color)
cv2.imshow('img_depth',img_depth)
# cv2.imshow('img_result', img_result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
_, src_bin = cv2.threshold(img_mask, 0, 255, cv2.THRESH_OTSU)

cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

dst = cv2.bitwise_and(img_color, img_color, mask = img_mask) 

blocks = []

for i in range(1, cnt): # 각각의 객체 정보에 들어가기 위해 반복문. 범위를 1부터 시작한 이유는 배경을 제외
    (x, y, w, h, area) = stats[i]
    cen_x, cen_y = map(int, centroids[i])

    # 노이즈 제거
    if area < 600:
        continue
    
    blocks.append([x,y,w,h,area])
    
    # cv2.rectangle(dst, (x, y, w, h), (0, 255, 255))
    # cv2.rectangle(img_color, (x, y, w, h), (0, 0, 255))
    # cv2.line(img_color, (cen_x, cen_y), (cen_x, cen_y), (0,0,255), 5)


In [21]:

cv2.imshow('src', img_mask)
# cv2.imshow('src_bin', src_bin)
cv2.imshow('dst', dst)
# cv2.imshow('img_color', img_color)
cv2.waitKey()
cv2.destroyAllWindows()

In [22]:
blocks

[]

In [23]:
img_mask*(labels==1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [24]:
(labels==1)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [25]:
binary_mask = (img_mask*(labels==1)//255)

In [26]:
(img_depth*binary_mask).shape

(480, 640)

In [27]:
get_block_depth = img_depth*binary_mask

In [28]:
cv2.imshow('block_depth', get_block_depth)

cv2.waitKey()
cv2.destroyAllWindows()